In [83]:
import torch
import torch.nn.functional as F
from torch import nn

In [103]:
class TestModel(nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        self.conv1a = nn.Conv2d(3, 16, (3, 3), padding=1)
        self.conv1b = nn.Conv2d(16, 16, (3, 3), padding=1)
        
        self.conv2a = nn.Conv2d(16, 32, (4, 4), padding=1, stride=2)
        self.conv2b = nn.Conv2d(32, 32, (3, 3), padding=1)
        
        self.conv3a = nn.Conv2d(32, 64, (4, 4), padding=1, stride=2)
        self.conv3b = nn.Conv2d(64, 64, (3, 3), padding=1)
        
        self.fc1 = self.fc1 = nn.Linear(64*12*12, 128)
        self.fc2 = nn.Linear(128, 1)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
#         print(x.shape)
        out = self.conv1a(x)
#         print(out.shape)
        out = F.relu(out)
        out = self.conv1b(out)
        out = F.relu(out)
        
        out = self.conv2a(out)
        out = F.relu(out)
        out = self.conv2b(out)
        out = F.relu(out)
        
        out = self.conv3a(out)
        out = F.relu(out)
        out = self.conv3b(out)
        out = F.relu(out)
        
#         print(out.shape)
        out = out.view(out.shape[0], -1)
        
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        
#         print(out)
        
        return out

In [6]:
# convert each image to tensor format
transform = transforms.Compose([
    transforms.ToTensor()  # convert to tensor
])

NameError: name 'transforms' is not defined

In [113]:
import pandas as pd
from torch.utils.data import Dataset
import cv2
from skimage import io
import numpy as np

data_path = '../Data'
labels_path = '{}/train_labels/train_labels.csv'.format(data_path)

train_labels = pd.read_csv(labels_path)

print(len(train_labels))

class TrainDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        img_name = '{}/{}.tif'.format(self.root_dir,
                                self.data_frame.iloc[idx, 0])
        img = cv2.imread(img_name)
        img = img[24:-24,24:-24,:]
        img = np.moveaxis(img, -1, 0)
        
        return torch.Tensor(img), self.data_frame.iloc[idx, 1].astype(float)

218


In [114]:
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchbearer import Trial
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

import time

dataset = TrainDataset(labels_path, '../Data/train')
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

# trainset = TrainDataset(labels_path, '../Data/train')
# train_loader = DataLoader(trainset, batch_size=12, shuffle=True)

In [115]:
model = TestModel()

model.train()

loss_function = nn.BCELoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(train_loader, test_generator=validation_loader)
trial.run(epochs=10)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

/Users/Dom/anaconda3/envs/cancer/lib/python3.7/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


{'test_loss': 0.6971936225891113, 'test_binary_acc': 0.34597155451774597}


/Users/Dom/anaconda3/envs/cancer/lib/python3.7/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


In [106]:
n_epochs = 1
n_batches = len(train_loader)

for epoch in range(n_epochs):
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0

    for i, data in enumerate(train_loader, 0):
        #Get inputs
        inputs, labels = data
#         inputs.unsqueeze_(0)

        #Wrap them in a Variable object
        inputs, labels = Variable(inputs), Variable(labels).float()

        #Set the parameter gradients to zero
        optimiser.zero_grad()
        
        #Forward pass, backward pass, optimize
        outputs = model(inputs)
#         print(labels)
        loss_size = loss_function(outputs, labels)
        loss_size.backward()
        optimiser.step()
        
#         print(loss_size)
        
        #Print statistics
        running_loss += loss_size.item()
        total_train_loss += loss_size.item()
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
            #Reset running loss and time
            running_loss = 0.0
            start_time = time.time()

Epoch 1, 10% 	 train_loss: 1.36 took: 0.18s
Epoch 1, 21% 	 train_loss: 2.79 took: 0.15s
Epoch 1, 31% 	 train_loss: 1.45 took: 0.17s
Epoch 1, 42% 	 train_loss: 1.40 took: 0.33s
Epoch 1, 52% 	 train_loss: 1.38 took: 0.20s
Epoch 1, 63% 	 train_loss: 1.19 took: 0.24s
Epoch 1, 73% 	 train_loss: 0.77 took: 0.24s
Epoch 1, 84% 	 train_loss: 2.18 took: 0.32s
Epoch 1, 94% 	 train_loss: 1.56 took: 0.19s
